# Advanced Problems in Computer Vision

So far typical computer vision problems revolved around object detection, classification and segmentation. But there are some other problems that require advanced techniques of computer vision to solve.

## Object Measurement

In many cases, we not only need to identify the object, we need to measure its dimensions or some other metric. For example we might need to create an application to measure the feet size based on image. In these type of use cases we cant just calculate the number of pixels. We need to have a proper methodology to accurately measure the values in varies scales.

One of the most simplest way to solve this problem is that having a reference object in the frame. If we can premeasure the size of that reference object, then we can use it to measure the unknown object dimensions once we detect it properly. 

For the detection work, first we can do a segmentation task using a Mask-RCNN like model. That way we can clearly separate the reference object and interested object from the background. Then we have to do a rotation correction to make sure the object we are trying to measure is in correct orientation. To do that we can use PCA (Principal Component Analysis) and get the eigen vectors for segmentation masks. Then the largest eigen vector magnitude is the size of object we are trying to measure in its longest direction. (This is because eigen vectors represent the field in orthogonal axes with their highest variance representations). 

Also since PCA axes are orthogonal and in a different space which have its axes aligned with object of interest's direction of greatest variance, we can use it to measure how much we need to rotate the original image to get the correct rotation.

The complexity of this task comes in combining above mentioned techniques in to a single working solution based on the usecase.


## Counting

Another interesting complex problem in computer vision is object counting. There are many applications that could be enhanced by solving this problem properly.

There are multiple ways to tackle this problem. Based on the use case we need to find a suitable method.

1. 